In [10]:
import numpy as np
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
import cv2
import os

# Функция для генерации изображений и масок
def generate_dataset(mesh_file, output_dir, angles, image_size=(256, 256)):
    # Проверка и создание директории для сохранения данных
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Загрузка STL файла
    your_mesh = mesh.Mesh.from_file(mesh_file)

    for i, angle in enumerate(angles):
        # Создание новой фигуры и осей для 3D визуализации
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        # Добавление объекта из STL файла на оси
        ax.add_collection3d(mplot3d.art3d.Poly3DCollection(your_mesh.vectors))

        # Масштабирование к размеру объекта
        scale = your_mesh.points.flatten()
        ax.auto_scale_xyz(scale, scale, scale)

        # Установка ракурса
        ax.view_init(elev=angle[0], azim=angle[1])

        # Отключение осей
        ax.axis('off')

        # Сохранение изображения
        image_path = os.path.join(output_dir, f'image_{i}.png')
        plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

        # Загрузка и изменение размера изображения
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image_resized = cv2.resize(image, image_size)

        # Применение порогового сегментирования для создания маски
        _, segmented = cv2.threshold(image_resized, 1, 255, cv2.THRESH_BINARY)

        # Сохранение маски
        mask_path = os.path.join(output_dir, f'mask_{i}.png')
        cv2.imwrite(mask_path, segmented)

        # Перезапись изображения в нужном разрешении
        cv2.imwrite(image_path, image_resized)

# Углы обзора (например, вид сверху, сбоку и т.д.)
angles = [(90, 90), (90, 0), (0, 90), (0, 0), (45, 45)]

# Путь к STL файлу и выходной директории
mesh_file = '../data/moon.stl'
output_dir = '../output'

# Генерация датасета
generate_dataset(mesh_file, output_dir, angles)
